In [1]:
!pip install git+https://github.com/openai/CLIP.git
!conda install av')

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-jo38tpk5
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-jo38tpk5
^C
ERROR: Operation cancelled by user
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [2]:
!nvidia-smi

Thu Nov 17 17:59:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:16:00.0 Off |                    0 |
| N/A   31C    P0    41W / 300W |      3MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torchvision.transforms as T
from PIL import Image
import torch
from torchvision.io.video import read_video,read_video_timestamps
import clip
from kaldiio import WriteHelper

In [4]:
def get_features(video_path, clip_model, preprocess_transforms, frame_rate, batch_size):
    
    
    step = int(v_fps//frame_rate)
    print(step)
    #Preprocess Images
    vid = vid[0: vid.shape[0]:step,::]
    features = torch.zeros((vid.shape[0], 768))
    
    num_frames = vid.shape[0] 
    feature_sets = []
    
    for i in range(0, num_frames, batch_size):        
        for j in range(i, i+batch_size):
            
                features = torch.zeros((vid.shape[0], 768))
                frame = vid[j]
                v_image = T.ToPILImage()(frame)
                v_processed = preprocess_transforms(v_image)
                v_processed = v_processed.unsqueeze(0).to('cuda')
                image_features = clip_model.encode_image(v_processed)
                features[i] = image_features
                
        feature_sets.append(features)
        
    return feature_sets

In [28]:
def get_features(start_frame,batch_size,video_length,clip_model,preprocess_transforms,vid):
    features = []
    for i in range(start_frame, start_frame+batch_size):
        if i> video_length:
            break
        
        frame = vid[i]
        v_image = T.ToPILImage()(frame)
        v_processed = preprocess_transforms(v_image)
        v_processed = v_processed.unsqueeze(0).to('cuda')
        image_features = clip_model.encode_image(v_processed)
        features.append(image_features)
    return torch.cat(features).cpu().detach().numpy()

In [21]:
torch.cuda.is_available()

True

In [48]:
def save_features(inpt_f,out_f, out_path, dir_path,frame_rate,video_path_eshani):
    
    with open(inp_f,mode='r') as f:
        
        with WriteHelper(
        
        f"ark,scp:{os.path.join(dir_path,out_f+'ark')},{out_path+'_'+'feature_set'+out_f+'scp'}") as writer:
            lines = f.readlines()
            for line in lines:
                #video_id, video_path = line.strip().split(" ")
                with torch.no_grad():
                    vid, _,video_metadata = read_video(video_path_eshani, pts_unit = 'sec',output_format = "TCHW")
                    _, v_fps = read_video_timestamps(video_path_eshani, pts_unit = 'sec')
                    step = int(v_fps//frame_rate)

                    vid = vid[0: vid.shape[0]:step,::]
                    if torch.cuda.is_available():
                        vid = vid.cuda()
                    #features = torch.zeros((vid.shape[0], 768))

                    num_frames = vid.shape[0]
                    batch_size = 2

                    for i in range(0, num_frames, batch_size):  
                        features_batch = get_features(i,batch_size,num_frames,model,preprocess,vid)
                        print(type(features_batch[0]))
                        writer("batch_"+str(int(i//batch_size)),features_batch)

In [13]:
!nvidia-smi

Thu Nov 17 18:00:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:16:00.0 Off |                    0 |
| N/A   32C    P0    54W / 300W |   1714MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
torch.cuda.clear_cache()

AttributeError: module 'torch.cuda' has no attribute 'clear_cache'

In [9]:
model, preprocess = clip.load("ViT-L/14")

In [10]:
video_path = '/ocean/projects/iri120008p/roshansh/corpora/how2/all_videos/-ImHtlpQoGI.mp4'
#features = get_features(0,12, 30, model, preprocess,) 

In [49]:
inp_f = '/ocean/projects/cis220078p/akapoor1/project/espnet/egs2/how2_2000h/sum1/dump/fbank_pitch/dev5_test_sum/wav.scp'
out_f = "feats"
out_path = "1"
dir_path = "./"
frame_rate = 6
import os
save_features(inp_f,out_f, out_path, dir_path,frame_rate,video_path)

<class 'numpy.ndarray'>


ValueError: Unsupported array type: float16

In [ ]:
#